In [81]:
import requests
import json
from bs4 import BeautifulSoup
import re
import psycopg2
import time
import numpy as np

In [21]:
link = 'http://www.whosampled.com/Usher/OMG/'
html = requests.get(link).content
soup = BeautifulSoup(html, 'html.parser')
print soup


<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge; chrome=1" http-equiv="X-UA-Compatible"/><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o?o:n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({QJf3ax:[function(e,n){function t(e){function n(n,t,a){e&&e(n,t,a),a||(a={});for(var u=c(n),f=u.length,s=i(a,o,r),p=0;f>p;p++)u[p].apply(s,t);return s}function a(e,n){f[e]=c(e).concat(n)}function c(e){return f[e]||[]}function u(){return t(n)}var f={};return{on:a,emit:n,create:u,listeners:c,_events:f}}function r(){return{}}var o="nr@context",i=e("gos");n.exports=t()},{gos:"7eSDFh"}],ee:[function(e,n){n.exports=e("QJ

In [11]:
def song_data(soup):
    url_base = 'http://www.whosampled.com'
    song = {}
    
    song['name'] = soup.select('h1[itemprop="name"]')[0].text
    print song['name']
    song['artists'] = [x.text for x in soup.select('h2 a')]
    song['artist_links'] = [url_base + x['href'] for x in soup.select('h2 a')]
    song['album'] = soup.select('.trackReleaseDetails h3')[0].text
    song['record_label'] = re.split('; |, |\*|\n', soup.select('.trackReleaseDetails h4')[0].text)
    song['year'] = soup.select('.trackReleaseDetails')[0].contents[4].strip()
    if len(soup.select('.trackReleaseDetails a')) > 0:
        song['producers'] = soup.select('.trackReleaseDetails a')[0].text
    else:
        song['producers'] = ''
    song['relationships'] = sample_data(soup)
    
    return song

In [12]:
def sample_data(soup):
    song_sample_data = {}
    for header in soup.select('.sectionHeader'):
        if header.text.find('samples') != -1:
            ul = header.find_next_sibling('ul')
            song_sample_data['sampled_from_song'] = [x.text for x in ul.select('.trackName')]
            song_sample_data['sampled_from_song_url'] = [x['href'] for x in ul.select('.trackName')]
            song_sample_data['sampled_from_artist'] = [x.text for x in ul.select('.trackArtist a')]
            song_sample_data['sampled_from_artist_url'] = [x['href'] for x in ul.select('.trackArtist a')]
        elif header.text.find('sampled') != -1:
            ul = header.find_next_sibling('ul')
            song_sample_data['sampled_by_song'] = [x.text for x in ul.select('.trackName')]
            song_sample_data['sampled_by_song_url'] = [x['href'] for x in ul.select('.trackName')]
            song_sample_data['sampled_by_artist'] = [x.text for x in ul.select('.trackArtist a')]
            song_sample_data['sampled_by_artist_url'] = [x['href'] for x in ul.select('.trackArtist a')]
        elif header.text.find('covered') != -1:
            ul = header.find_next_sibling('ul')
            song_sample_data['covered_by_song'] = [x.text for x in ul.select('.trackName')]
            song_sample_data['covered_by_song_url'] = [x['href'] for x in ul.select('.trackName')]
            song_sample_data['covered_by_artist'] = [x.text for x in ul.select('.trackArtist a')]
            song_sample_data['convered_by_artist_url'] = [x['href'] for x in ul.select('.trackArtist a')]
        elif header.text.find('remixed') != -1:
            ul = header.find_next_sibling('ul')
            song_sample_data['remixed_by_song'] = [x.text for x in ul.select('.trackName')]
            song_sample_data['remixed_by_song_url'] = [x['href'] for x in ul.select('.trackName')]
            song_sample_data['remixed_by_artist'] = [x.text for x in ul.select('.trackArtist a')]
            song_sample_data['remixed_by_artist_url'] = [x['href'] for x in ul.select('.trackArtist a')]
    return song_sample_data

In [22]:
song = song_data(soup)

print song['album']
print song['record_label']
print song['year']
print song['producers']
print song

OMG
[u'Usher', u'will.i.am'] Raymond v. Raymond [u'LaFace'] 2010 will.i.am
Raymond v. Raymond
[u'LaFace']
2010
will.i.am
{'album': u'Raymond v. Raymond', 'record_label': [u'LaFace'], 'name': u'OMG', 'relationships': {'remixed_by_song_url': [u'/remix/72189/Usher-will.i.am-OMG-(Riva-Starr-Remix)-Usher-will.i.am-OMG/', u'/remix/72190/Usher-will.i.am-OMG-(Almighty-Mix)-Usher-will.i.am-OMG/'], 'sampled_by_artist_url': [u'/Girl-Talk/', u'/DJ-Earworm/', u'/DJ-Earworm/'], 'covered_by_artist': [u'Audiogroove'], 'remixed_by_artist': [u'Riva Starr', u'Usher', u'will.i.am'], 'sampled_from_artist_url': [u'/Deep-Purple/', u'/Black-Eyed-Peas/'], 'sampled_from_song': [u'Speed King (Live)', u'Boom Boom Pow'], 'sampled_by_artist': [u'Girl Talk', u'DJ Earworm', u'DJ Earworm'], 'remixed_by_song': [u'OMG (Riva Starr Remix)', u'OMG (Almighty Mix)'], 'covered_by_song': [u'OMG (Oh My Gosh)'], 'sampled_from_artist': [u'Deep Purple', u'Black Eyed Peas'], 'sampled_by_song_url': [u'/sample/68913/Girl-Talk-Down-fo

In [14]:
def get_soup(url):
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [15]:
def get_song_urls_from_artist(artist):
    url_base = 'http://www.whosampled.com'
    song_urls = []
    url = url_base + '/' + artist
    soup = get_soup(url)
    pages = int(soup.select('.page')[-1].text)
    for pg_num in xrange(pages):
        url = url_base + '/' + artist + '/?sp=' + str(pg_num+1)
        soup = get_soup(url)
        current_pg_urls = [url_base + song['href'] for song in soup.select('.trackName a[itemprop="url"]')]
        song_urls += current_pg_urls
    return song_urls

In [16]:
get_song_urls_from_artist('Drake')

[u'http://www.whosampled.com/Drake/0-to-100-the-Catch-Up/',
 u'http://www.whosampled.com/Drake/Pound-CakeParis-Morton-Music-2/',
 u'http://www.whosampled.com/Nicki-Minaj/Truffle-Butter/',
 u'http://www.whosampled.com/Kendrick-Lamar/Poetic-Justice/',
 u'http://www.whosampled.com/Drake/Started-From-the-Bottom/',
 u'http://www.whosampled.com/Drake/Know-Yourself/',
 u'http://www.whosampled.com/Drake/Take-Care/',
 u'http://www.whosampled.com/Drake/Legend/',
 u'http://www.whosampled.com/Drake/6-God/',
 u'http://www.whosampled.com/Drake/Fancy/',
 u'http://www.whosampled.com/The-Game/100/',
 u'http://www.whosampled.com/Drake/Jungle/',
 u'http://www.whosampled.com/A$AP-Rocky/Fuckin%27-Problems/',
 u'http://www.whosampled.com/Drake/Miss-Me/',
 u'http://www.whosampled.com/Drake/Cameras-Good-Ones-Go-Interlude/',
 u'http://www.whosampled.com/DJ-Khaled/I%27m-on-One/',
 u'http://www.whosampled.com/PARTYNEXTDOOR/Over-Here/',
 u'http://www.whosampled.com/Drake/How-Bout-Now/',
 u'http://www.whosampled.c

In [ ]:
for header in soup.select('.sectionHeader'):
    print header.text
    ul = header.find_next_sibling('ul')
    print ul.select('.trackName')[0].text

In [17]:
def scrape_Drake(artist):
    all_data = []
    artist = '-'.join(artist.split())
    urls = get_song_urls_from_artist(artist)
    for url in urls:
        soup = get_soup(url)
        all_data.append(song_data(soup))
    return all_data

In [18]:
data = scrape_Drake('Chance the Rapper')

Favorite Song
[u'Chance the Rapper', u'Childish Gambino'] Acid Rap [u''] 2013 Nate Fox
Lost
[u'Chance the Rapper', u'Noname Gypsy'] Acid Rap [u''] 2013 Nate Fox
NaNa
[u'Chance the Rapper', u'Action Bronson'] Acid Rap [u''] 2013 brandUn DeShay
Juice
[u'Chance the Rapper'] Acid Rap [u''] 2013 Nate Fox
Pusha Man
[u'Chance the Rapper', u'Nate Fox', u'Lili K'] Acid Rap [u'-'] 2013 CeeJ
Everybody's Something
[u'Chance the Rapper', u'Saba', u'BJ the Chicago Kid'] Acid Rap [u''] 2013 DJ Ozone
Good Ass Intro
[u'Chance the Rapper'] Acid Rap [u''] 2013 
Wendy N Becky
[u'Joey Bada$$', u'Chance the Rapper']  [u'Cinematic Music Group'] 2013 Thelonious Martin
Chain Smoker
[u'Chance the Rapper'] Acid Rap [u'Mixtape'] 2013 Nate Fox
Long Time
[u'Chance the Rapper'] 10 Day [u'SaveMoney'] 2012 
22 Offs
[u'Chance the Rapper'] 10 Day [u'Not on Label'] 2012 Flying Lotus
14,400 Minutes
[u'Chance the Rapper'] 10 Day [u'Not on Label'] 2012 DJ SuchNSuch
Everything's Good (Good Ass Outro)
[u'Chance the Rapper'] A

In [20]:
len(data)

48

# Data architecture

###Table Artist:
artist_url - string
artist_name - string

###Table Producer:
producer_url - string
producer_name - string

###Table Song:
song_url - string
song_name - string
album_name - string
label - string
year - int
main_genre - string

###Table ArtistToSong:
artist_url - string
song_url - string

###Table ProducerToSong:
producer_url - string
song_url - string

###Table SongRelationship:
song1 - string
song2 - string
relationship - int (1 for sampled from, 2 for sampled by, 3 for cover of, 4 for covered by, 5 for remix of, and 6 for remixed by)

In [175]:
class GetData(object):
    def __init__(self):
        self.url_base = 'http://www.whosampled.com'
        self.used_genres = set([])
        self.unused_genres = set(['/genre/Hip-Hop/', '/genre/Electronic-Dance/', '/genre/Rock-Pop/', \
                                '/genre/Soul-Funk-Disco/', '/genre/Jazz-Blues/', '/genre/Reggae/', \
                                '/genre/Country-Folk/', '/genre/World/', '/genre/Soundtrack/', '/genre/Classical/'])
        self.used_artists = set([])
        self.unused_artists = set([])
        self.used_producers = set([])
        self.unused_producers = set([])
        self.used_songs = set([])
        self.unused_songs = set([])

    # Create soup object
    def _get_soup(self, url):
        '''
        INPUT: url string
        OUTPUT: soup object for processing
        '''
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'html.parser')
        return soup

    # Collect all artist names based on genre
    def _get_artists_by_genre(self, genre):
        '''
        INPUT: genre string
        OUTPUT: saves artists names into SQL database
        '''
        artists = []
        url = self.url_base + genre
        genre_soup = self._get_soup(url)
        alpha_urls = [x['href'] for x in genre_soup.select('.browse-letters a')]
        for alphabet_url in alpha_urls:
            pages = int(self._get_num_pages(alphabet_url))
            decomp_url = alphabet_url.split('/')
            for page in xrange(pages):
                decomp_url[-2] = str(page + 1)
                page_url = '/'.join(decomp_url)
                artists += self._get_link_and_text_data(page_url, '.artistName a')
        
        conn = None
        conn = psycopg2.connect(database = 'beatlineage', user = 'gSchool', host = '/tmp')
        cur = conn.cursor()
        
        for artist in artists:
            if artist[0] in self.used_artists:
                continue
            else:
                self.unused_artists.add(artist[0])
                cur.execute('INSERT INTO artists VALUES(%s, %s)', artist)
                conn.commit()
                
        if conn:
            cur.close()
            conn.close()
    
    # Collect all song names based on artist
    def _get_songs_by_artist(self, artist):
        '''
        INPUT: artist string
        OUTPUT: saves songs into list
        '''
        songs = []
        pages = int(self._get_num_pages(artist))
        for pg_num in xrange(pages):
            page_url = artist + '?sp=' + str(pg_num + 1)
            songs += self._get_link_and_text_data(page_url, '.trackName a[itemprop="url"]')
        
        for song in songs:
            if song[0] in self.used_songs:
                continue
            else:
                self.unused_songs.add(song[0])

    # Collect all data for a single song
    def _get_song_data(self, song):
        '''
        INPUT: song string
        OUTPUT: saves producer and song data into SQL database
        '''
        song_data = []
        producers = []
        sample_data = {}
        url = self.url_base + song
        soup = self._get_soup(url)
        
        # Basic song info
        name = soup.select('h1[itemprop="name"]')[0].text
        artists = [x['href'] for x in soup.select('h2 a')]
        album = soup.select('.trackReleaseDetails h3')[0].text
        main_label = re.split('; |, |\*|\n', soup.select('.trackReleaseDetails h4')[0].text)[0]
        year = int(soup.select('.trackReleaseDetails')[0].contents[4].strip())
        if len(soup.select('.trackReleaseDetails a')) > 0:
            producers = self._get_link_and_text_data(song, '.trackReleaseDetails a')
        else:
            producers = ''        
               
        print url
        print name
        print artists
        print album
        print main_label
        print year
        print producers
        
        # Song sampling history
        for header in soup.select('.sectionHeader'):
            if header.select('.moreButton'):
                more_url = header.select('.moreButton')[0]['href']
                pages = int(self._get_num_pages(more_url))
                for pg_num in xrange(pages):
                    page_url = more_url + '?sp=' + str(pg_num + 1)
                    for header in soup.select('.sectionHeader'):
                        self._get_sample_data(header)
            else:
                self._get_sample_data(header)
        
        
        '''
        # Open a connection to postgresql
        conn = None
        conn = psycopg2.connect(database = 'beatlineage', user = 'gSchool', host = '/tmp')
        cur = conn.cursor()
        
        # Write to song to artist relationship table
        cur.execute('INSERT INTO songs VALUES(%s, %s, %s, %s, %s, %s)', (song[0], song[1], '', '', 0, ''))
        conn.commit()
        
        
        
        # Close connection to postgresql
        if conn:
            cur.close()
            conn.close()
        '''

    # Get the sampling data
    def _get_sample_data(self, header):
        '''
        INPUT: song string
        OUTPUT: sampling data
        '''
        sampled_from_song_url = []
        sampled_by_song_url = []
        cover_of_song_url = []
        covered_by_song_url = []
        remix_of_song_url = []
        remixed_by_song_url = []
        
        ul = header.find_next_sibling('ul')
        if header.text.find('samples of') != -1:
            genre, sampled_from_song_url = self._get_genre_and_song_url(ul, 0)
        elif header.text.find('sampled in') != -1:
            genre, sampled_by_song_url = self._get_genre_and_song_url(ul, 1)
        elif header.text.find('cover of') != -1:
            genre, cover_of_song_url = self._get_genre_and_song_url(ul, 0)
        elif header.text.find('covered in') != -1:
            genre, covered_by_song_url = self._get_genre_and_song_url(ul, 1)
        elif header.text.find('remix of') != -1:
            genre, remix_of_song_url = self._get_genre_and_song_url(ul, 0)
        elif header.text.find('remixed in') != -1:
            genre, remixed_by_song_url = self._get_genre_and_song_url(ul, 1)
        else:
            pass
        
        print genre
        print sampled_from_song_url
        print sampled_by_song_url
        print cover_of_song_url
        print covered_by_song_url
        print remix_of_song_url
        print remixed_by_song_url
    
    # Uses the urls from _get_sample_data to grab the original song's genre and the new song's url
    # Side is binary for the side of the original song, 0 for left (sample of, cover of, remix of) and
    # 1 for right (sampled in, covered in, remixed in)
    def _get_genre_and_song_url(self, ul, side):
        '''
        INPUT: unordered list object and the side of the original song
        OUTPUT: original song genre and list of new song urls
        '''
        url_list = [x['href'] for x in ul.select('.trackName')]
        song_urls = []
        for url in url_list:
            pg_url = self.url_base + url
            soup = self._get_soup(pg_url)
            genres = [x['href'] for x in soup.select('.sampleAdditionalInfoContainer a[href^="/genre"]')]
            original_song_genre = genres[side]
            new_song_genre = genres[not side]
            
            song_urls += [x['href'] for x in soup.select('h3 .trackName')][0]
            
            if new_song_genre in self.used_genres:
                continue
            else:
                self.unused_genres.add(new_song_genre)
            
            return original_song_genre, song_urls
    
    # Get the number of pages on a page
    def _get_num_pages(self, end_url):
        '''
        INPUT: ending url string
        OUTPUT: number of pages of request
        '''
        url = self.url_base + end_url
        page_soup = self._get_soup(url)
        try:
            num_of_pg = int(page_soup.select('.page')[-1].text)
        except:
            num_of_pg = 1
        return num_of_pg
    
    # Extract text and link data for a given CSS tag
    def _get_link_and_text_data(self, end_url, tag):
        '''
        INPUT: ending url and CSS tag
        OUTPUT: list of tuple of (text, link)
        '''
        print end_url
        url = self.url_base + end_url
        page_soup = self._get_soup(url)
        text = [x.text for x in page_soup.select(tag)]
        link = [x['href'] for x in page_soup.select(tag)]
        return zip(link, text)
    
    def _get_songs(self):
        genre = '/genre/Soul-Funk-Disco/'
        self._get_artists_by_genre(genre)
        while len(self.unused_artists) > 0:
            artist = self.unused_artists.pop()
            self._get_songs_by_artist(artist)
            self.used_artists.add(artist)

In [176]:
data = GetData()

In [96]:
con = None
con = psycopg2.connect(database = 'beatlineage', user = 'gSchool', host = '/tmp')
cur = con.cursor()
con.autocommit = True

In [97]:
# Create artist table in SQL
def create_artist_table(cur, con):
    try:
        cur.execute('CREATE TABLE artists (artist_url varchar, \
                                           artist_name varchar)')
    except:
        pass

    # Example entry:
    #cur.execute('INSERT INTO artists VALUES(%s, %s)', ('/Kanye-West/', 'Kanye West'))

create_artist_table(cur, con)

In [98]:
# Create song table in SQL
def create_song_table(cur, con):
    try:
        cur.execute('CREATE TABLE songs (song_url varchar, \
                                         song_name varchar, \
                                         album_name varchar, \
                                         label_name varchar, \
                                         year integer, \
                                         main_genre varchar)')
    except:
        pass

    # Example entry:
    #cur.execute('INSERT INTO songs VALUES(%s, %s, %s, %s, %s, %s)', ('/Lil-Jon/Lovers-%26-Friends/', 'Lovers & Friends', 'Crunk Juice', 'TVT', 2004, '/genre/Hip-Hop/'))

create_song_table(cur, con)

In [80]:
data._get_songs()

/browse/artists/Soul-Funk-Disco/W/1/
/browse/artists/Soul-Funk-Disco/W/2/
/browse/artists/Soul-Funk-Disco/W/3/
/browse/artists/Soul-Funk-Disco/W/4/
/browse/artists/Soul-Funk-Disco/W/5/
/browse/artists/Soul-Funk-Disco/W/6/
/browse/artists/Soul-Funk-Disco/W/7/
/browse/artists/Soul-Funk-Disco/W/8/
/browse/artists/Soul-Funk-Disco/W/9/
/browse/artists/Soul-Funk-Disco/W/10/
/browse/artists/Soul-Funk-Disco/W/11/
/browse/artists/Soul-Funk-Disco/W/12/
/browse/artists/Soul-Funk-Disco/W/13/
/William-Weatherspoon/?sp=1
/Wing-and-a-Prayer-Fife-and-Drum-Corps/?sp=1
/Warren-Moore/?sp=1
/We-Productions/?sp=1
/William-Stevenson/?sp=1
/William-Stevenson/?sp=2
/Willy-Albimoor/?sp=1
/William-%22Mickey%22-Stevenson/?sp=1
/Willie-Henderson/?sp=1
/Warren-Schatz/?sp=1
/Wilkins/?sp=1
/Willie-Tee/?sp=1
/Wynder-K.-Frog/?sp=1
/Walter-Murphy/?sp=1
/Walter-Murphy/?sp=2
/Wild-Sugar/?sp=1
/William-Salter/?sp=1
/Willie-Clarke/?sp=1
/Willie-Clarke/?sp=2
/Winston-Johnson/?sp=1
/Waylon/?sp=1
/Whiskey-Barons/?sp=1
/White-

In [84]:
url = 'http://www.whosampled.com/Black-Eyed-Peas/The-Time-(Dirty-Bit)/'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')
re.split('; |, |\*|\n', soup.select('.trackReleaseDetails h4')[0].text)

[u'Interscope']

In [177]:
data._get_song_data('/Black-Eyed-Peas/Boom-Boom-Pow/')

/Black-Eyed-Peas/Boom-Boom-Pow/
http://www.whosampled.com/Black-Eyed-Peas/Boom-Boom-Pow/
Boom Boom Pow
[u'/Black-Eyed-Peas/']
The E.N.D.
A&M
2009
[(u'/will.i.am/', u'will.i.am')]
/genre/Hip-Hop/
[u'/', u'B', u'l', u'a', u'c', u'k', u'-', u'E', u'y', u'e', u'd', u'-', u'P', u'e', u'a', u's', u'/', u'B', u'o', u'o', u'm', u'-', u'B', u'o', u'o', u'm', u'-', u'P', u'o', u'w', u'/']
[]
[]
[]
[]
[]
/genre/Hip-Hop/
[u'/', u'B', u'l', u'a', u'c', u'k', u'-', u'E', u'y', u'e', u'd', u'-', u'P', u'e', u'a', u's', u'/', u'B', u'o', u'o', u'm', u'-', u'B', u'o', u'o', u'm', u'-', u'P', u'o', u'w', u'/']
[]
[]
[]
[]
[]
/genre/Hip-Hop/
[]
[u'/', u'U', u's', u'h', u'e', u'r', u'/', u'O', u'M', u'G', u'/']
[]
[]
[]
[]
/genre/Hip-Hop/
[]
[]
[]
[u'/', u'R', u'u', u'c', u'k', u'a', u'-', u'R', u'u', u'c', u'k', u'a', u'-', u'A', u'l', u'i', u'/', u'C', u'h', u'i', u'n', u'g', u'-', u'C', u'h', u'a', u'n', u'g', u'-', u'C', u'h', u'o', u'n', u'g', u'/']
[]
[]
/genre/Hip-Hop/
[]
[]
[]
[]
[]
[u'/', u'B', u

In [146]:
url = 'http://www.whosampled.com/sample/64343/Black-Eyed-Peas-Boom-Boom-Pow-Daft-Punk-Harder,-Better,-Faster,-Stronger/'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')
print [x['href'] for x in soup.select('.sampleAdditionalInfoContainer a[href^="/genre"]')]
print [x['href'] for x in soup.select('h3 .trackName')]
#[x['href'] for x in soup.select('.sampleAdditionalInfoContainer')]

[u'/genre/Hip-Hop/', u'/genre/Electronic-Dance/']
[u'/Black-Eyed-Peas/Boom-Boom-Pow/', u'/Daft-Punk/Harder,-Better,-Faster,-Stronger/']
